# Enrich LVMPD Calls for Service Data
We are going to enrich the LVMPD Calls for Service data with attributes from a few sources:
1. The LVMPD Area Commands - So we can show which area command the call for service took place in
2. The LVMPD Sector Beats - So we can see which sector and which beat the call for service took place in
3. Census Block - So we can see which census block/blockgroup/tract/etc. the call for service took place in, in case we want to join to any census data in the future.

## Import any packages we'll be using
In this program, we will use the __[geopandas](https://geopandas.org/en/stable/)__ package for spatial data manipulation and will use the merge command from the __[pandas](https://pandas.pydata.org/)__ package.

In [29]:
import geopandas
import pandas as pd
import csv

## Load the shape files with the attributes we want to append.
LVMPD Area command shapefile: https://opendata-lvmpd.hub.arcgis.com/datasets/0803e2a7a8e44517b6eb9aa8071df996_0/explore </br>
LVMPD Sector Beat shapefile: https://opendata-lvmpd.hub.arcgis.com/datasets/48516778560747e28d69be39813157a9_0/explore </br>
Nevada Census shapefile: https://www2.census.gov/geo/tiger/TIGER2023/TABBLOCK20/tl_2023_32_tabblock20.zip

In [22]:
path_to_area_command = "Data/LVMPD/LVMPD_Area_Commands/AREA_COMMANDS.geojson"
gdfAreaCommand = geopandas.read_file( path_to_area_command ).to_crs( "EPSG:4326" )

gdfAreaCommand

,OBJECTID,NAME,AGENCY,AC,FULLNAME,geometry
0,1,DTAC,LVMPD,DT,DOWNTOWN AREA COMMAND,"POLYGON ((-115.10722 36.18540, -115.10722 36.1..."
1,2,NEAC,LVMPD,NE,NORTHEAST AREA COMMAND,"MULTIPOLYGON (((-115.12247 36.24696, -115.1235..."
2,3,SEAC,LVMPD,SE,SOUTHEAST AREA COMMAND,"MULTIPOLYGON (((-114.78665 36.00896, -114.7872..."
3,4,SCAC,LVMPD,SC,SOUTH CENTRAL AREA COMMAND,"POLYGON ((-115.10990 36.14410, -115.10988 36.1..."
4,5,CCAC,LVMPD,CC,CONVENTION CENTER AREA COMMAND,"POLYGON ((-115.15719 36.14364, -115.15633 36.1..."
5,6,EAC,LVMPD,EA,ENTERPRISE AREA COMMAND,"MULTIPOLYGON (((-115.18353 35.93215, -115.1835..."
6,7,SAC,LVMPD,SA,SUMMERLIN AREA COMMAND,"POLYGON ((-115.32493 36.21839, -115.32475 36.2..."
7,8,SVAC,LVMPD,SV,SPRING VALLEY AREA COMMAND,"POLYGON ((-115.20506 36.14451, -115.20431 36.1..."
8,9,BAC,LVMPD,BA,BOLDEN AREA COMMAND,"POLYGON ((-115.15178 36.20312, -115.15178 36.2..."
9,10,NWAC,LVMPD,NW,NORHTWEST AREA COMMAND,"POLYGON ((-115.32063 36.33621, -115.32063 36.3..."


In [23]:
path_to_sector_beat = "Data/LVMPD/LVMPD_Sector_Beat/LVMPD_Sector_Beat.geojson"
gdfSectorBeat = geopandas.read_file( path_to_sector_beat ).to_crs( "EPSG:4326" )

gdfSectorBeat

,OBJECTID,BEAT,SECTOR,ACOM,geometry
0,1,81,8,RO,"POLYGON ((-114.57339 35.20050, -114.57243 35.1..."
1,2,82,8,RO,"MULTIPOLYGON (((-114.80763 35.31603, -114.8076..."
2,3,83,8,RO,"MULTIPOLYGON (((-115.30501 35.63433, -115.3050..."
3,4,84,8,RO,"POLYGON ((-115.63762 36.18852, -115.62215 36.1..."
4,5,85,8,RO,"POLYGON ((-115.34896 36.33607, -115.34896 36.3..."
...,...,...,...,...,...
91,92,X4,X,NW,"POLYGON ((-115.20667 36.29127, -115.20667 36.2..."
92,93,X5,X,NW,"POLYGON ((-115.33212 36.33621, -115.33079 36.3..."
93,94,X6,X,NW,"POLYGON ((-115.36886 36.39561, -115.36883 36.3..."
94,95,Y9,Y,NLV,"POLYGON ((-115.13793 36.18843, -115.13821 36.1..."


In [24]:
path_to_census = "Data/Census Geographies/tl_2023_32_tabblock20"
gdfCensus = geopandas.read_file( path_to_census ).to_crs( "EPSG:4326" )

gdfCensus

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,GEOIDFQ20,NAME20,MTFCC20,UR20,UACE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry
0,32,013,010501,2371,320130105012371,1010000US320130105012371,Block2371,G5040,R,None,S,179003245,3405,+41.1677818,-117.7822288,5,6,"POLYGON ((-117.86910 41.17824, -117.86573 41.1..."
1,32,007,951700,2113,320079517002113,1010000US320079517002113,Block2113,G5040,R,None,S,22267753,18682,+41.7792872,-115.6577947,1,1,"POLYGON ((-115.70946 41.76841, -115.70921 41.7..."
2,32,007,951700,2467,320079517002467,1010000US320079517002467,Block2467,G5040,R,None,S,279739070,15541,+41.8169420,-115.2016102,0,0,"POLYGON ((-115.39868 41.71237, -115.39806 41.7..."
3,32,007,951700,3155,320079517003155,1010000US320079517003155,Block3155,G5040,R,None,S,193947493,14058,+41.5922701,-114.6188375,0,0,"POLYGON ((-114.71471 41.56799, -114.71434 41.5..."
4,32,007,951700,2238,320079517002238,1010000US320079517002238,Block2238,G5040,R,None,S,45364224,401927,+41.5705022,-116.7469853,0,0,"POLYGON ((-116.80298 41.59337, -116.80294 41.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57404,32,031,003002,1007,320310030021007,1010000US320310030021007,Block1007,G5040,U,74179,S,10318,0,+39.5352917,-119.7439756,0,0,"POLYGON ((-119.74456 39.53575, -119.74339 39.5..."
57405,32,003,005868,1015,320030058681015,1010000US320030058681015,Block1015,G5040,U,47995,S,6926,0,+35.9982490,-115.2508924,7,37,"POLYGON ((-115.25162 35.99847, -115.25018 35.9..."
57406,32,007,951700,2858,320079517002858,1010000US320079517002858,Block2858,G5040,R,None,S,8230433,0,+41.2663998,-116.8117228,3,2,"POLYGON ((-116.83235 41.25909, -116.83233 41.2..."
57407,32,003,003621,1000,320030036211000,1010000US320030036211000,Block1000,G5040,U,47995,S,59648,0,+36.3056674,-115.1715723,57,93,"POLYGON ((-115.17650 36.30595, -115.17111 36.3..."


## Loop through the 5 Calls for Service files, enrich the data, and export

In [31]:
years = ['2019', '2020', '2021', '2022', '2023']
for x in years:
    # Load
    print( "Starting to process year: " + x )
    path_to_data = "Data/LVMPD/LVMPD Calls For Service " + x + "/LVMPD_Calls_For_Service_" + x + ".geojson"
    print( "\tLoading file: " + path_to_data )
    gdfCallsForService = geopandas.read_file( path_to_data ).to_crs( "EPSG:4326" )
    print( "\tFile loaded." )

    # Joins
    print( "\tJoining to Area Commands." )
    gdfCallsForService_w_AreaCommands = gdfCallsForService.sjoin( gdfAreaCommand, how = "left", predicate = "within" )
    print("\tJoining to Sector Beats.")
    gdfCallsForService_w_SectorBeats = gdfCallsForService.sjoin( gdfSectorBeat, how = "left", predicate = "within" )
    print( "\tJoining to Census." )
    gdfCallsForService_w_Census = gdfCallsForService.sjoin( gdfCensus, how = "left", predicate = "within" )

    # Merges
    print( "\tMerging Calls for Service with Area Commands." )
    gdfCallsForService_Merged = pd.merge( gdfCallsForService, gdfCallsForService_w_AreaCommands[['OBJECTID_left', 'NAME', 'AGENCY', 'AC', 'FULLNAME']], how = "left", left_on = "OBJECTID", right_on = "OBJECTID_left" )
    gdfCallsForService_Merged = gdfCallsForService_Merged.drop( 'OBJECTID_left', axis = 1)
    
    print( "\tMerging with Sector Beats." )
    gdfCallsForService_Merged = pd.merge(gdfCallsForService_Merged, gdfCallsForService_w_SectorBeats[['OBJECTID_left', 'BEAT', 'SECTOR', 'ACOM']], how = "left", left_on = "OBJECTID", right_on = "OBJECTID_left" )
    gdfCallsForService_Merged = gdfCallsForService_Merged.drop( 'OBJECTID_left', axis = 1 )
    
    print( "\tMerging with Census." )
    gdfCallsForService_Merged = pd.merge( gdfCallsForService_Merged, gdfCallsForService_w_Census[['OBJECTID', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20', 'GEOIDFQ20']], how = "left", left_on = "OBJECTID", right_on = "OBJECTID" )

    # Filter the columns to just what we're interested in. The 2023 file capitalizes IncidentNumber differently...
    if x == '2023':
        fnlDF = gdfCallsForService_Merged[['OBJECTID', 'IncidentNumber', 'NAME', 'AGENCY', 'AC', 'FULLNAME', 'BEAT', 'SECTOR', 'ACOM', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20', 'GEOIDFQ20']]
    else:
       fnlDF = gdfCallsForService_Merged[['OBJECTID', 'incidentnumber', 'NAME', 'AGENCY', 'AC','FULLNAME', 'BEAT', 'SECTOR', 'ACOM', 'STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20', 'GEOIDFQ20']]
    
    # Export
    path_to_write = 'Data/Enriched/LVMPD_Calls_For_Service_' + x + '_Enriched.csv'
    
    print( "\tExporting file to:" + path_to_write )
    fnlDF.to_csv( path_to_write, quoting = csv.QUOTE_NONNUMERIC, index = False )
    
    print( "\tFinished with year: " + x )

Starting to process year: 2019
	Loading file: Data/LVMPD/LVMPD Calls For Service 2019/LVMPD_Calls_For_Service_2019.geojson
	File loaded.
	Joining to Area Commands.
	Joining to Sector Beats.
	Joining to Census.
	Merging Calls for Service with Area Commands.
	Merging with Sector Beats.
	Merging with Census.
	Exporting file to:Data/LVMPD/Enriched/LVMPD_Calls_For_Service_2019_Enriched.csv
	Finished with year: 2019
Starting to process year: 2020
	Loading file: Data/LVMPD/LVMPD Calls For Service 2020/LVMPD_Calls_For_Service_2020.geojson
	File loaded.
	Joining to Area Commands.
	Joining to Sector Beats.
	Joining to Census.
	Merging Calls for Service with Area Commands.
	Merging with Sector Beats.
	Merging with Census.
	Exporting file to:Data/LVMPD/Enriched/LVMPD_Calls_For_Service_2020_Enriched.csv
	Finished with year: 2020
Starting to process year: 2021
	Loading file: Data/LVMPD/LVMPD Calls For Service 2021/LVMPD_Calls_For_Service_2021.geojson
	File loaded.
	Joining to Area Commands.
	Joining

c:\Users\Alex\anaconda3\envs\GeoEnv\Lib\site-packages\geopandas\io\file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


	File loaded.
	Joining to Area Commands.
	Joining to Sector Beats.
	Joining to Census.
	Merging Calls for Service with Area Commands.
	Merging with Sector Beats.
	Merging with Census.
	Exporting file to:Data/LVMPD/Enriched/LVMPD_Calls_For_Service_2023_Enriched.csv
	Finished with year: 2023
